In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="""다음 주제와 관련된 한국의 유명한 장소나 활동 5가지를 알려주세요.
주제: {topic}

{format_instructions}""",
    input_variables=["topic"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

result = chain.invoke({"topic": "음식"})
print(result)  # ['명동 칼국수', '부산 돼지국밥', ...]


['서울의 한정식 거리', '부산의 자갈치 시장', '광주의 궁전국밥', '대전의 성심당', '제주도의 흑돼지 거리']


In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException
from enum import Enum
from pprint import pprint

# ✅ 환경변수 로드
load_dotenv()

# ✅ Groq LLM 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)

# ✅ 감정 클래스 정의
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# ✅ 출력 파서 설정
enum_parser = EnumOutputParser(enum=Sentiment)
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)

# ✅ 포맷 지시사항
format_instructions = enum_parser.get_format_instructions()
print("🧾 감정 분류 포맷:\n", format_instructions)

# ✅ 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

# ✅ 체인 구성
prompt = ChatPromptTemplate.from_template(template).partial(
    format_instructions=format_instructions
)

chain = prompt | llm | fixing_parser

# ✅ 테스트 입력
test_reviews = [
    "이 영화 정말 재미없었어요. 시간 아까웠습니다.",
    "배우들의 연기가 훌륭하고, 음악도 감동적이었어요!",
    "그냥 무난한 영화였네요. 나쁘지도, 좋지도 않았어요.",
    "이건 최고의 영화야!!!!👍",
]

# ✅ 실행
for review in test_reviews:
    print(f"\n📝 리뷰: {review}")
    try:
        result = chain.invoke({"text": review})
        print("🧠 분석 결과:", result.value)
    except OutputParserException as e:
        print("❌ 파싱 실패:", str(e))


🧾 감정 분류 포맷:
 Select one of the following options: 긍정, 부정, 중립

📝 리뷰: 이 영화 정말 재미없었어요. 시간 아까웠습니다.
🧠 분석 결과: 부정

📝 리뷰: 배우들의 연기가 훌륭하고, 음악도 감동적이었어요!
🧠 분석 결과: 긍정

📝 리뷰: 그냥 무난한 영화였네요. 나쁘지도, 좋지도 않았어요.
🧠 분석 결과: 중립

📝 리뷰: 이건 최고의 영화야!!!!👍
🧠 분석 결과: 긍정


In [3]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: list[str] = Field(description="학생의 취미")
    goal: str = Field(description="학생의 목표")

parser = PydanticOutputParser(pydantic_object=StudentInfo)

format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
아래 자기소개를 읽고, 이름, 나이, 전공, 취미 리스트, 목표를 JSON 형식으로 추출해 주세요.

자기소개:
{intro}

{format_instructions}
""",
    input_variables=["intro"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | llm | parser

intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

result = chain.invoke({"intro": intro_text})
print(result)


name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'


In [11]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

# ✅ 환경변수 로드
load_dotenv()

# ✅ Groq LLM 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# ✅ 스키마 정의
schemas = [
    ResponseSchema(name="destination", description="여행한 장소"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="만족도 점수 (1~5점)"),
    ResponseSchema(name="activities", description="여행에서 한 활동 리스트")
]

parser = StructuredOutputParser(response_schemas=schemas)
format_instructions = parser.get_format_instructions()

# ✅ 프롬프트
prompt = PromptTemplate(
    template="""아래 여행 이야기를 읽고, 여행지, 기간, 예산, 만족도, 활동 리스트를 구조화된 JSON 형식으로 출력하세요.

여행 후기:
{story}

{format_instructions}
""",
    input_variables=["story"],
    partial_variables={"format_instructions": format_instructions}
)

# ✅ 체인 연결
chain = prompt | llm | parser

# ✅ 테스트 입력
story = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

result = chain.invoke({"story": story})
print(result)


{'destination': '부산', 'duration': '2박 3일', 'budget': '30만원', 'rating': '4', 'activities': ['해운대 바다 구경', '자갈치시장에서 회 먹기', '감천문화마을 구경']}
